[参考链接](https://www.appcoda.com.tw/core-ml-3/)

In [54]:
!pip install coremltools

In [55]:
import os
import coremltools

# 設定剛剛下載的 model 的位置
coreml_model_path = "./MNISTClassifier.mlmodel"

# 讀取 model 的 spec，關於 model 的所有資訊都會被存在這邊
spec = coremltools.utils.load_spec(coreml_model_path)

# 將 classifier 存下來方便接下來利用
classifier = spec.neuralNetworkClassifier

In [56]:
def printLayers(layers):
    for (i, layer) in enumerate(layers):
        print(" layer index = %d\ttype: %s, input: %s, output: %s" %(i-len(layers), layer.WhichOneof('layer'), layer.input[0], layer.output[0]))

In [57]:
printLayers(classifier.layers)

 layer index = -14	type: convolution, input: image, output: drawing_conv0_fwd
 layer index = -13	type: activation, input: drawing_conv0_fwd, output: drawing_conv0_relu_fwd
 layer index = -12	type: pooling, input: drawing_conv0_relu_fwd, output: drawing_pool0_fwd
 layer index = -11	type: convolution, input: drawing_pool0_fwd, output: drawing_conv1_fwd
 layer index = -10	type: activation, input: drawing_conv1_fwd, output: drawing_conv1_relu_fwd
 layer index = -9	type: pooling, input: drawing_conv1_relu_fwd, output: drawing_pool1_fwd
 layer index = -8	type: convolution, input: drawing_pool1_fwd, output: drawing_conv2_fwd
 layer index = -7	type: activation, input: drawing_conv2_fwd, output: drawing_conv2_relu_fwd
 layer index = -6	type: pooling, input: drawing_conv2_relu_fwd, output: drawing_pool2_fwd
 layer index = -5	type: flatten, input: drawing_pool2_fwd, output: drawing_flatten0_reshape0
 layer index = -4	type: innerProduct, input: drawing_flatten0_reshape0, output: drawing_dense0_fwd

In [58]:
numberOfChannels = classifier.layers[-2].innerProduct.inputChannels

In [59]:
for output in spec.description.output:
    output_name = output.name
    output_type = output.type.WhichOneof('Type')

    print(f"Output name: {output_name}")
    print(f"Output type: {output_type}")

    # 根据输出类型，打印输出形状
    if output_type == 'multiArrayType':
        shape = output.type.multiArrayType.shape
        print(f"Output shape: {shape}")
    elif output_type == 'imageType':
        width = output.type.imageType.width
        height = output.type.imageType.height
        print(f"Output image size: {width}x{height}")
    elif output_type == 'dictionaryType':
        key_type = output.type.dictionaryType.WhichOneof('KeyType')
        print(f"Output is a dictionary with keys of type: {key_type}")
    # 可以添加其他输出类型的处理逻辑

Output name: labelProbabilities
Output type: dictionaryType
Output is a dictionary with keys of type: int64KeyType
Output name: classLabel
Output type: int64Type


In [60]:
del classifier.layers[-1]
del classifier.layers[-1]

In [61]:
# 印出 model 的類型
print(spec.WhichOneof('Type'))

# neuralNetworkClassifier

neuralNetworkClassifier


In [62]:
# 把修改過的 layer 加到 neuralNetwork 裡
spec.neuralNetwork.layers.extend(classifier.layers)

# 移除掉 neuralNetworkClassfier 裡的 layers
del spec.neuralNetworkClassifier.layers[:]

# 把原本 preprocessing 的部份也複製到 neuralNetwork 裡
spec.neuralNetwork.preprocessing.extend(spec.neuralNetworkClassifier.preprocessing)

In [63]:
# 1
del spec.description.output[-1]
del spec.description.output[-1]

# 2
new_output = spec.description.output.add()
new_output.name = 'features'
new_output.shortDescription = 'Penultimate layer output'

new_output_params = new_output.type.multiArrayType
new_output_params.dataType = coremltools.proto.FeatureTypes_pb2.ArrayFeatureType.FLOAT32

# 3
new_output_params.shape.extend([numberOfChannels])

# 4
spec.neuralNetwork.layers[-1].output[0] = "features"

In [64]:
print(spec.WhichOneof('Type'))

# neuralNetwork

neuralNetwork


In [65]:
for output in spec.description.output:
    output_name = output.name
    output_type = output.type.WhichOneof('Type')

    print(f"Output name: {output_name}")
    print(f"Output type: {output_type}")

    # 根据输出类型，打印输出形状
    if output_type == 'multiArrayType':
        shape = output.type.multiArrayType.shape
        print(f"Output shape: {shape}")
    elif output_type == 'imageType':
        width = output.type.imageType.width
        height = output.type.imageType.height
        print(f"Output image size: {width}x{height}")
    elif output_type == 'dictionaryType':
        key_type = output.type.dictionaryType.WhichOneof('KeyType')
        print(f"Output is a dictionary with keys of type: {key_type}")
    # 可以添加其他输出类型的处理逻辑

Output name: features
Output type: multiArrayType
Output shape: [128]


In [66]:
from coremltools.models.nearest_neighbors import KNearestNeighborsClassifierBuilder

knn_builder = KNearestNeighborsClassifierBuilder(input_name='features',
                                                 output_name='label',
                                                 number_of_dimensions=numberOfChannels,
                                                 default_class_label='unknown',
                                                 k=5,
                                                 weighting_scheme='inverse_distance',
                                                 index_type='linear')

# 只是設定描述
knn_builder.description = 'Classifies ' + str(numberOfChannels) + ' dimension vector based on 5 nearest neighbors'
knn_spec = knn_builder.spec
knn_spec.description.input[0].shortDescription = 'Feature vector from CNN'
knn_spec.description.output[0].shortDescription = 'Predicted label. Defaults to \'unknown\''
knn_spec.description.output[1].shortDescription = 'Probabilities / score for each possible label.'

In [67]:
pipeline_spec = coremltools.proto.Model_pb2.Model()
pipeline_spec.specificationVersion = coremltools._MINIMUM_UPDATABLE_SPEC_VERSION
pipeline_spec.isUpdatable = True

In [68]:
# 讓 pipeline classifier 的 input 描述跟前面做好的 feature extractor 一樣，並且加上一些跟 training 相關的描述
pipeline_spec.description.input.extend(spec.description.input[:])
pipeline_spec.description.trainingInput.extend([spec.description.input[0]])
pipeline_spec.description.trainingInput[0].shortDescription = 'Example drawing'
pipeline_spec.description.trainingInput.extend([knn_spec.description.output[0]])
pipeline_spec.description.trainingInput[1].shortDescription = 'Label for the example drawing'

# 讓 pipeline classifier 的 output 描述跟 k-NN 的 output 一致
pipeline_spec.description.output.extend(knn_spec.description.output[:])
pipeline_spec.description.predictedFeatureName = knn_spec.description.predictedFeatureName
pipeline_spec.description.predictedProbabilitiesName = knn_spec.description.predictedProbabilitiesName

# 一些額外的描述
pipeline_spec.description.metadata.author = 'A Developer'
pipeline_spec.description.metadata.license = 'A License'
pipeline_spec.description.metadata.shortDescription = ('An updatable model which can be used to train a tiny 28 x 28 drawing classifier based on user examples.'
                                                       ' It uses a drawing embedding model trained on the MINST)')

In [69]:
# feature extractor
pipeline_spec.pipelineClassifier.pipeline.models.add().CopyFrom(spec)

# k-NN
pipeline_spec.pipelineClassifier.pipeline.models.add().CopyFrom(knn_spec)

# 把 model 存下來
updatable_mlmodel = coremltools.models.MLModel(pipeline_spec)
updatable_mlmodel.save("./UpdatableDrawingClassifier.mlmodel")

In [70]:
# 加载模型
updated_ml = coremltools.models.MLModel('./UpdatableDrawingClassifier.mlmodel')

# 获取模型规格
updated_spec = updated_ml.get_spec()

# 打印是否支持更新的信息
is_updatable = updated_spec.isUpdatable if hasattr(updated_spec, 'isUpdatable') else 'Unknown'
print(f"Model is updatable: {is_updatable}")

Model is updatable: True


In [71]:


# 检查模型是否为 pipeline 类型
if updated_spec.WhichOneof('Type') == 'pipelineClassifier':
    # 遍历 pipeline 中的每个模型
    for model_spec in updated_spec.pipelineClassifier.pipeline.models:
        print(f"Model is updatable: {model_spec.isUpdatable}")
        # 检查模型类型并打印所有层
        if model_spec.WhichOneof('Type') == 'neuralNetwork':
            printLayers(model_spec.neuralNetwork.layers)
        elif model_spec.WhichOneof('Type') == 'neuralNetworkClassifier':
            printLayers(model_spec.neuralNetworkClassifier.layers)
        elif model_spec.WhichOneof('Type') == 'neuralNetworkRegressor':
            printLayers(model_spec.neuralNetworkRegressor.layers)
        else:
            print(model_spec.WhichOneof('Type'))
            print("Sub-model is not a neural network model.")
else:
    print("Model is not a pipeline model.")


Model is updatable: False
 layer index = -12	type: convolution, input: image, output: drawing_conv0_fwd
 layer index = -11	type: activation, input: drawing_conv0_fwd, output: drawing_conv0_relu_fwd
 layer index = -10	type: pooling, input: drawing_conv0_relu_fwd, output: drawing_pool0_fwd
 layer index = -9	type: convolution, input: drawing_pool0_fwd, output: drawing_conv1_fwd
 layer index = -8	type: activation, input: drawing_conv1_fwd, output: drawing_conv1_relu_fwd
 layer index = -7	type: pooling, input: drawing_conv1_relu_fwd, output: drawing_pool1_fwd
 layer index = -6	type: convolution, input: drawing_pool1_fwd, output: drawing_conv2_fwd
 layer index = -5	type: activation, input: drawing_conv2_fwd, output: drawing_conv2_relu_fwd
 layer index = -4	type: pooling, input: drawing_conv2_relu_fwd, output: drawing_pool2_fwd
 layer index = -3	type: flatten, input: drawing_pool2_fwd, output: drawing_flatten0_reshape0
 layer index = -2	type: innerProduct, input: drawing_flatten0_reshape0, ou